In [21]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as score
import warnings
warnings.filterwarnings("ignore")
customer_churn=pd.read_csv("Telecom_customer_churn.csv")

In [2]:
df=pd.DataFrame(data=customer_churn)

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [5]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [6]:
df.loc[df.Churn=='No','Churn'] = 0 
df.loc[df.Churn=='Yes','Churn'] = 1

In [7]:
df.groupby("gender").Churn.mean()

gender
Female    0.269209
Male      0.261603
Name: Churn, dtype: float64

In [8]:
df.tenure.describe()

count    7043.000000
mean       32.371149
std        24.559481
min         0.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dummy_columns = []
for column in df.columns:
    if df[column].dtype == object and column != 'customerID':
        if df[column].nunique() == 2:
            df[column] = le.fit_transform(df[column]) 
        else:
            dummy_columns.append(column)

In [11]:
df = pd.get_dummies(data = df,columns = dummy_columns)

In [12]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,Churn,...,TotalCharges_995.35,TotalCharges_996.45,TotalCharges_996.85,TotalCharges_996.95,TotalCharges_997.65,TotalCharges_997.75,TotalCharges_998.1,TotalCharges_999.45,TotalCharges_999.8,TotalCharges_999.9
0,7590-VHVEG,0,0,1,0,1,0,1,29.85,0,...,0,0,0,0,0,0,0,0,0,0
1,5575-GNVDE,1,0,0,0,34,1,0,56.95,0,...,0,0,0,0,0,0,0,0,0,0
2,3668-QPYBK,1,0,0,0,2,1,1,53.85,1,...,0,0,0,0,0,0,0,0,0,0
3,7795-CFOCW,1,0,0,0,45,0,0,42.30,0,...,0,0,0,0,0,0,0,0,0,0
4,9237-HQITU,0,0,0,0,2,1,1,70.70,1,...,0,0,0,0,0,0,0,0,0,0


In [16]:
x = df.drop(['Churn','customerID'],axis=1).values
y = df.Churn.values.reshape(-1,1)

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.18,random_state=42)

In [19]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5775, 6570)
(1268, 6570)
(5775, 1)
(1268, 1)


In [20]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score


In [23]:
def rfc():
    rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1, )
    rfc.fit(x_train, y_train)
    pred = rfc.predict(x_test)
    print('Accuracy in percent = ',score(pred, y_test)*100)
    print(f1_score(y_test, pred, average='macro'))
    print(f1_score(y_test, pred, average='micro'))
    print(f1_score(y_test, pred, average='weighted'))

In [24]:
rfc()

Accuracy in percent =  79.49526813880126
0.7072602442931375
0.7949526813880127
0.7828223154498166


In [27]:
def lr():
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    pred = lr.predict(x_test)
    print('Accuracy in percent = ',score(pred, y_test)*100)
    print(f1_score(y_test, pred, average='macro'))
    print(f1_score(y_test, pred, average='micro'))
    print(f1_score(y_test, pred, average='weighted'))
    


In [28]:
lr()

Accuracy in percent =  82.72870662460568
0.7684941179805302
0.8272870662460567
0.8235147487517114


In [29]:
from sklearn.externals import joblib
joblib.dump(lr,"lrsave")

['lrsave']

In [32]:
import pickle
s=pickle.dumps(lr)
clf2=pickle.loads(s)

In [33]:
clf2

<function __main__.lr()>